In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

在Vertex AI上使用预构建容器为PyTorch图像模型提供服务

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/prediction/pytorch_image_classification_with_prebuilt_serving_containers.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fprediction%2Fpytorch_image_classification_with_prebuilt_serving_containers.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/prediction/pytorch_image_classification_with_prebuilt_serving_containers.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/prediction/pytorch_image_classification_with_prebuilt_serving_containers.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

**_注意_**：此笔记本已在以下环境中进行了测试：

* Python 版本 = 3.9

## 概述

本教程演示了如何使用预先构建的服务容器上传和部署PyTorch图像模型，以及如何进行在线和批量预测。

Vertex AI提供了用于提供训练模型工件预测和解释的预构建容器。使用预构建容器通常比为预测创建自定义容器更简单。

了解更多关于[用于预测的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)。

### 目标

在本教程中，您将学习如何使用预构建的Vertex AI容器和TorchServe打包和部署PyTorch图像分类模型，以用于提供在线和批量预测。

本教程使用以下Google Cloud ML服务和资源：

- `Vertex AI模型注册表`
- `Vertex AI模型`资源
- `Vertex AI端点`资源

执行的步骤包括：

- 从PyTorch下载预训练图像模型
- 创建自定义模型处理程序
- 将模型工件打包到模型存档文件中
- 上传模型进行部署
- 部署模型进行预测
- 进行在线预测
- 进行批量预测

模型

本教程使用了来自PyTorch TorchVision的预训练图像模型[resnet18](https://pytorch.org/vision/master/models/generated/torchvision.models.resnet18.html)。

### 成本

本教程使用了 Google Cloud 的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI 定价](https://cloud.google.com/vertex-ai/pricing)，
以及 [云存储定价](https://cloud.google.com/storage/pricing)，
并使用 [定价计算器](https://cloud.google.com/products/calculator/)，
根据您预计的使用情况生成成本估算。

开始吧

安装Python的Vertex AI SDK和其他所需的软件包。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform==1.23.0 \
                                 tensorflow==2.12.0 \
                                 torch==2.0.0 \
                                 torchvision==0.15.1 \
                                 torch-model-archiver==0.7.1 \
                                 packaging==14.3

重新启动运行时（仅适用于Colab）

要使用新安装的软件包，您必须重新启动Google Colab上的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning"> <b>⚠️内核将重新启动。在继续下一步之前，请等待完成。⚠️</b> </div>

### 验证您的笔记本环境（仅适用于Colab）

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置 Google Cloud 项目信息，并初始化 Python 的 Vertex AI SDK

要开始使用 Vertex AI，您必须拥有现有的 Google Cloud 项目并[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储桶

创建一个存储桶来存储诸如数据集等中间工件。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在： 运行以下单元以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

#### 初始化用于Python的Vertex AI SDK

为您的项目初始化Vertex AI SDK以便在Python中使用。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### 导入库

In [ ]:
import base64
import json
import os
import pathlib
import urllib.request

import tensorflow as tf
import torch
from PIL import Image
from torchvision import models

下载预训练的图像模型[resnet18](https://pytorch.org/vision/master/models/generated/torchvision.models.resnet18.html)从PyTorch TorchVision。

In [ ]:
# Create a local directory for model artifacts
model_path = "model"

!rm -r $model_path
!mkdir $model_path

model_name = "resnet-18-custom-handler"
model_file = f"{model_path}/{model_name}.pt"

In [ ]:
# Use scripted mode to save the PyTorch model locally
model = models.resnet18(pretrained=True)
script_module = torch.jit.script(model)
script_module.save(model_file)

创建自定义模型处理程序

自定义模型处理程序是一个Python脚本，当您使用模型归档工具时，您可以将其与模型一起打包。该脚本通常定义如何预处理输入数据，调用模型和后处理输出。

TorchServe为`image_classifier`、`image_segmenter`、`object_detector`和`text_classifier`提供了[默认处理程序](https://pytorch.org/serve/default_handlers.html)。在本教程中，您将创建一个自定义处理程序，扩展默认的[`image_classifier`](https://github.com/pytorch/serve/blob/master/ts/torch_handler/image_classifier.py)处理程序。

In [ ]:
hander_file = f"{model_path}/custom_handler.py"

In [ ]:
%%writefile {hander_file}

import torch
import torch.nn.functional as F
from ts.torch_handler.image_classifier import ImageClassifier
from ts.utils.util import map_class_to_label


class CustomImageClassifier(ImageClassifier):

    # Only return the top 3 predictions
    topk = 3

    def postprocess(self, data):
        ps = F.softmax(data, dim=1)
        probs, classes = torch.topk(ps, self.topk, dim=1)
        probs = probs.tolist()
        classes = classes.tolist()
        return map_class_to_label(probs, self.mapping, classes)

## 下载一个index_to_name.json文件

PyTorch的`image_classifier`、`text_classifier`和`object_detector`都可以自动将数字类别（0,1,2...）映射为友好的字符串。要实现这一点，只需在您的模型归档文件中包含一个包含类别号码到友好名称映射的`index_to_name.json`文件。

In [ ]:
index_to_name_file = f"{model_path}/index_to_name.json"

urllib.request.urlretrieve(
    "https://github.com/pytorch/serve/raw/master/examples/image_classifier/index_to_name.json",
    index_to_name_file,
)

将模型产物打包在一个模型存档文件中

你可以使用[`Torch模型归档器`](https://github.com/pytorch/serve/tree/master/model-archiver)将所有模型产物打包在一个模型存档文件中。

请注意，预构建的PyTorch服务容器需要模型存档文件命名为`model.mar`，因此在`torch-model-archiver`命令中需要将模型名称设置为`model`。

In [ ]:
# Add torch-model-archiver to the PATH
os.environ["PATH"] = f'{os.environ.get("PATH")}:~/.local/bin'

In [ ]:
!torch-model-archiver -f \
  --model-name model \
  --version 1.0  \
  --serialized-file $model_file \
  --handler $hander_file \
  --extra-files $index_to_name_file \
  --export-path $model_path

将模型构件复制到云存储

接下来，使用 `gsutil` 将模型构件复制到您的云存储存储桶中。

In [ ]:
MODEL_URI = f"{BUCKET_URI}/{model_name}"

!gsutil rm -r $MODEL_URI
!gsutil cp -r $model_path $MODEL_URI
!gsutil ls -al $MODEL_URI

上传模型进行部署

接下来，您将上传模型至`Vertex AI模型注册表`，这将为您的模型创建一个`Vertex AI模型`资源。本教程使用PyTorch v1.11容器，但对于您自己的用例，您可以从[PyTorch预构建容器列表](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers#pytorch)中进行选择。

In [ ]:
DEPLOY_IMAGE_URI = "us-docker.pkg.dev/vertex-ai/prediction/pytorch-cpu.1-11:latest"

deployed_model = aiplatform.Model.upload(
    display_name=model_name,
    serving_container_image_uri=DEPLOY_IMAGE_URI,
    artifact_uri=MODEL_URI,
)

部署模型进行预测

接下来，部署您的模型进行在线预测。您可以设置变量`DEPLOY_COMPUTE`来配置机器类型，用于进行预测所需的[计算资源](https://cloud.google.com/vertex-ai/docs/predictions/configure-compute)。

In [ ]:
DEPLOY_COMPUTE = "n1-standard-4"

endpoint = deployed_model.deploy(
    deployed_model_display_name=model_name,
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=None,
    accelerator_count=0,
)

## 进行在线预测

### 下载图像数据集
在此示例中，您可以使用 TensorFlow 的花卉数据集作为在线和批处理预测的输入。

In [ ]:
data_dir = tf.keras.utils.get_file(
    "flower_photos",
    origin="https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz",
    untar=True,
)

data_dir = pathlib.Path(data_dir)
images_files = list(data_dir.glob("daisy/*"))

获取在线预测

您将包含编码输入图像数据的`predict`请求发送到`endpoint`，并获得预测结果。

In [ ]:
with open(images_files[0], "rb") as f:
    data = {"data": base64.b64encode(f.read()).decode("utf-8")}

response = endpoint.predict(instances=[data])

In [ ]:
prediction = response.predictions[0]
prediction = dict(sorted(prediction.items(), key=lambda item: item[1], reverse=True))

print(prediction)
image = Image.open(images_files[0])
image

## 进行批量预测

### 创建批量输入文件

您需以JSONL格式创建批量输入文件，并将该文件存储在您的云存储桶中。

了解更多关于[输入数据要求](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions#input_data_requirements)。

In [ ]:
TEST_IMAGE_SIZE = 2
test_image_list = []
for i in range(TEST_IMAGE_SIZE):
    test_image_list.append(str(images_files[i]))

gcs_input_uri = f"{BUCKET_URI}/test_images.json"

with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    for test_image in test_image_list:
        with open(test_image, "rb") as image_f:
            data = {"data": base64.b64encode(image_f.read()).decode("utf-8")}
            f.write(json.dumps(data) + "\n")

提交一个批量预测任务

In [ ]:
JOB_DISPLAY_NAME = f"{model_name}_batch_predict_job_unique"

batch_predict_job = deployed_model.batch_predict(
    job_display_name=JOB_DISPLAY_NAME,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="jsonl",
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
)

### 获取批量预测

批处理作业完成后，结果将写入您在批处理请求中指定的Cloud Storage输出存储桶。您可以调用方法`iter_outputs()`获取包含结果的每个Cloud Storage文件的列表。

In [ ]:
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_files = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction.results"):
        prediction_files.append(blob.name)

In [ ]:
prediction_file = prediction_files[0]

results = []
gfile_name = f"{BUCKET_URI}/{prediction_file}"
with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
    for line in gfile.readlines():
        results.append(json.loads(line))

# Take one result as an example and print out the prediction.
prediction = results[0]["prediction"]
prediction = dict(sorted(prediction.items(), key=lambda item: item[1], reverse=True))
print(prediction)

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
endpoint.undeploy_all()
endpoint.delete()

deployed_model.delete()
batch_predict_job.delete()

delete_bucket = False
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI